In [32]:
import os 
import pickle
import random
import numpy as np

In [35]:
from gensim.models import Word2Vec

In [54]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras import backend as k
from tensorflow.keras.layers import Lambda
from tensorflow import expand_dims
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Flatten
import tensorflow

In [62]:
data_dir = os.path.join(os.getcwd(), 'glassdoor_problem')

with open(os.path.join(data_dir, 'training_data.pkl'), 'rb') as f:
    training_data = pickle.load(f)
    
with open(os.path.join(data_dir, 'validation_data.pkl'), 'rb') as f:
    validation_data = pickle.load(f)
    
with open(os.path.join(data_dir, 'test_data.pkl'), 'rb') as f:
    test_data = pickle.load(f)

In [64]:
# To keep training fast, it will good to keep batches of data during training than just one data point at a time
def cluster_sentences(data, batch_size) : 
    """
    Given: 
    data:[("word11 word12 ..", label_vector), ("word21 word22 ..", label_vector), ....] - List of sentences with labels
    batch_size : size of the batch to be used during training
    
    Return:
    A cluster of TOKENIZED sentences, which is a list of list, where each entry in the output is a list of length batch_size 
    containing tokenized sentences with same number of words shown below
    [[([word11, word12], label_vector), ([word21, word22], label_vector), ...], 
    [([word31, word32, word33], label_vector), ([word41, word42, word43], label_vector), ...],
    ]
    """
    tokenized_sentences = [(sen[0].split(" "), sen[1]) for sen in data]
    
    # list at ith position below will store sentences with i words
    store_same_length_sentences = [[] for i in range(100)]
    _ = [store_same_length_sentences[len(sentence[0])].append(sentence) for sentence in tokenized_sentences]
    
    # take only those sentence lengths which have number of sentences greater than batch_size 
    valid_sentence_lengths = [i for i in range(100) if len(store_same_length_sentences[i])>batch_size]
    
    batched_sentences = []
    for length in valid_sentence_lengths : 
        same_length_sentences = store_same_length_sentences[length]
        number_of_same_length_sentences = len(same_length_sentences)
        number_of_training_batches = int(number_of_same_length_sentences/batch_size)
        _ = random.shuffle(same_length_sentences)
        batches = [same_length_sentences[(i*batch_size):((i+1)*batch_size)] for i in range(number_of_training_batches)]
        batched_sentences = batched_sentences + batches 
    
    return batched_sentences

In [65]:
batch_size = 8 
training_clusters = cluster_sentences(training_data, batch_size)
validation_clusters = cluster_sentences(validation_data, batch_size)
test_clusters = cluster_sentences(test_data, batch_size)

In [66]:
# load word2vec model 
wordvec_model = Word2Vec.load(os.path.join(os.getcwd(), 'glassdoor_problem/wordvecmodel'))

def get_word_vector(word, word2vecmodel) : 
    return word2vecmodel.wv[word]

# a function to create the input matrix given the tokenized sentences using the word2vec model
def get_word2vec_input_matrix(list_of_words) : 
    """
    List of list of words
    Each entry in list_of_words is a list of words. All entries are of the same length
    """
    store_all_together = []
    for word_group in list_of_words : 
        store_all_together.append(np.array([get_word_vector(word, wordvec_model) for word in word_group]))
    return np.array(store_all_together)

In [67]:
"""
This cell has
Generator to load the data batch wise
A Lambda function to get the last elements of a batch of data to be sent to CNN part
"""
def generator(data):
    """
    The generator to load data for training
    This will take batched sentences and load each batch in every iteration
    """
    L = len(data)
    batch_size = 1 
    while True:
        batch_start = 0 
        batch_end = batch_size
        while batch_start < L:
            x_portion = [sen[0] for sen in data[batch_start:batch_end][0]]
            y_portion = np.array([sen[1] for sen in data[batch_start:batch_end][0]])
            x_values = get_word2vec_input_matrix(x_portion)
            y_values = y_portion
            yield x_values, y_values
            batch_start = batch_end
            batch_end = batch_start + batch_size 

def get_last_elements(tensor) : 
    last_words = []
    for i in range(tensor.shape[0]) : 
        last_word_representation = tensor[i][-1]
        expanded = expand_dims(last_word_representation, axis=0)
        expanded = tensorflow.reshape(expanded, (30, 1))
        last_words.append(expanded)
    return tensorflow.convert_to_tensor(last_words)

# batch_size to train
batch_size = 8 
# size of the word vectors to be passed in the neural network model
word_vec_size = 100

inp = Input(batch_shape=(batch_size, None, word_vec_size))
encoded1 = LSTM(30, return_sequences=True, activation='tanh')(inp)
encoded = Lambda(lambda x: get_last_elements(x))(encoded1)
convolved = Conv1D(32, 2, input_shape=(1, 30), activation='relu')(encoded)
pooled = MaxPooling1D(3, strides=3)(convolved)
flattened = Flatten()(pooled)
output_probabilities = Dense(8, activation='sigmoid')(flattened)
output_vector = Lambda(lambda x: x*8)(output_probabilities)
model = Model(inp, output_vector)
model.compile(loss='mean_squared_error', optimizer='sgd')

In [68]:
#callback to save checkpoints
filepath= os.path.join(os.getcwd(), 'glassdoor_problem/glassdoor_assignment_models/') + "weights-improvement-{epoch:02d}-{val_loss:02f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')

In [69]:
model.fit_generator(generator(training_clusters), steps_per_epoch=len(training_clusters), epochs=400, 
                        validation_data=generator(validation_clusters),
                        validation_steps=len(validation_clusters),  verbose=1, callbacks=[checkpoint])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 588 steps, validate for 61 steps
Epoch 1/400
585/588 [============================>.] - ETA: 0s - loss: 7.3416
Epoch 00001: val_loss improved from inf to 6.83667, saving model to /Users/purushottamsinha/Desktop/glassdoor_problem/glassdoor_assignment_models/weights-improvement-01-6.836670.h5
588/588 [==============================] - 7s 13ms/step - loss: 7.3433 - val_loss: 6.8367
Epoch 2/400
585/588 [============================>.] - ETA: 0s - loss: 5.0239
Epoch 00002: val_loss improved from 6.83667 to 4.07904, saving model to /Users/purushottamsinha/Desktop/glassdoor_problem/glassdoor_assignment_models/weights-improvement-02-4.079044.h5
588/588 [==============================] - 4s 8ms/step - loss: 5.0266 - val_loss: 4.0790
Epoch 3/400
583/588 [============================>.] - ETA: 0s - loss: 3.1662
Epoch 00003: val_loss improved from 4.07904 to 3.67924, saving model to /Users/purushottamsinha/Desktop/glassdoor_problem/glassd

Epoch 30/400
584/588 [============================>.] - ETA: 0s - loss: 1.2063
Epoch 00030: val_loss did not improve from 1.63836
588/588 [==============================] - 4s 7ms/step - loss: 1.2147 - val_loss: 1.6473
Epoch 31/400
583/588 [============================>.] - ETA: 0s - loss: 1.2155
Epoch 00031: val_loss did not improve from 1.63836
588/588 [==============================] - 4s 7ms/step - loss: 1.2327 - val_loss: 1.7374
Epoch 32/400
587/588 [============================>.] - ETA: 0s - loss: 1.1824
Epoch 00032: val_loss did not improve from 1.63836
588/588 [==============================] - 4s 7ms/step - loss: 1.1875 - val_loss: 2.0125
Epoch 33/400
585/588 [============================>.] - ETA: 0s - loss: 1.1559
Epoch 00033: val_loss did not improve from 1.63836
588/588 [==============================] - 4s 7ms/step - loss: 1.1629 - val_loss: 1.7526
Epoch 34/400
584/588 [============================>.] - ETA: 0s - loss: 1.1528
Epoch 00034: val_loss did not improve from 1.

Epoch 64/400
583/588 [============================>.] - ETA: 0s - loss: 0.7760
Epoch 00064: val_loss did not improve from 1.45730
588/588 [==============================] - 4s 8ms/step - loss: 0.7906 - val_loss: 1.9034
Epoch 65/400
583/588 [============================>.] - ETA: 0s - loss: 0.7779
Epoch 00065: val_loss did not improve from 1.45730
588/588 [==============================] - 4s 6ms/step - loss: 0.7850 - val_loss: 1.6805
Epoch 66/400
587/588 [============================>.] - ETA: 0s - loss: 0.7969
Epoch 00066: val_loss did not improve from 1.45730
588/588 [==============================] - 4s 6ms/step - loss: 0.7991 - val_loss: 1.6639
Epoch 67/400
587/588 [============================>.] - ETA: 0s - loss: 0.7774
Epoch 00067: val_loss did not improve from 1.45730
588/588 [==============================] - 4s 6ms/step - loss: 0.7794 - val_loss: 1.5012
Epoch 68/400
584/588 [============================>.] - ETA: 0s - loss: 0.7827
Epoch 00068: val_loss did not improve from 1.

Epoch 99/400
584/588 [============================>.] - ETA: 0s - loss: 0.6539
Epoch 00099: val_loss did not improve from 1.40518
588/588 [==============================] - 4s 6ms/step - loss: 0.6555 - val_loss: 1.5367
Epoch 100/400
584/588 [============================>.] - ETA: 0s - loss: 0.6269
Epoch 00100: val_loss did not improve from 1.40518
588/588 [==============================] - 4s 6ms/step - loss: 0.6292 - val_loss: 1.5550
Epoch 101/400
586/588 [============================>.] - ETA: 0s - loss: 0.6320
Epoch 00101: val_loss did not improve from 1.40518
588/588 [==============================] - 4s 6ms/step - loss: 0.6338 - val_loss: 1.6610
Epoch 102/400
587/588 [============================>.] - ETA: 0s - loss: 0.6559
Epoch 00102: val_loss did not improve from 1.40518
588/588 [==============================] - 4s 6ms/step - loss: 0.6576 - val_loss: 1.5205
Epoch 103/400
585/588 [============================>.] - ETA: 0s - loss: 0.6674
Epoch 00103: val_loss did not improve fro

Epoch 136/400
587/588 [============================>.] - ETA: 0s - loss: 0.5520
Epoch 00136: val_loss did not improve from 1.35614
588/588 [==============================] - 4s 6ms/step - loss: 0.5535 - val_loss: 1.4227
Epoch 137/400
587/588 [============================>.] - ETA: 0s - loss: 0.5535
Epoch 00137: val_loss did not improve from 1.35614
588/588 [==============================] - 4s 6ms/step - loss: 0.5543 - val_loss: 1.4908
Epoch 138/400
585/588 [============================>.] - ETA: 0s - loss: 0.5534
Epoch 00138: val_loss did not improve from 1.35614
588/588 [==============================] - 4s 6ms/step - loss: 0.5547 - val_loss: 1.4459
Epoch 139/400
586/588 [============================>.] - ETA: 0s - loss: 0.5572
Epoch 00139: val_loss did not improve from 1.35614
588/588 [==============================] - 4s 6ms/step - loss: 0.5571 - val_loss: 1.4320
Epoch 140/400
585/588 [============================>.] - ETA: 0s - loss: 0.5437
Epoch 00140: val_loss improved from 1.35

...

587/588 [============================>.] - ETA: 0s - loss: 0.4713
Epoch 00247: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4727 - val_loss: 1.4110
Epoch 248/400
586/588 [============================>.] - ETA: 0s - loss: 0.4726
Epoch 00248: val_loss did not improve from 1.35089
588/588 [==============================] - 3s 6ms/step - loss: 0.4728 - val_loss: 1.4062
Epoch 249/400
586/588 [============================>.] - ETA: 0s - loss: 0.4702
Epoch 00249: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4704 - val_loss: 1.3953
Epoch 250/400
583/588 [============================>.] - ETA: 0s - loss: 0.4685
Epoch 00250: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 7ms/step - loss: 0.4694 - val_loss: 1.4634
Epoch 251/400
585/588 [============================>.] - ETA: 0s - loss: 0.4722
Epoch 00251: val_loss did not improve from 1.35089
588

587/588 [============================>.] - ETA: 0s - loss: 0.4594
Epoch 00284: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4603 - val_loss: 1.4193
Epoch 285/400
586/588 [============================>.] - ETA: 0s - loss: 0.4603
Epoch 00285: val_loss did not improve from 1.35089
588/588 [==============================] - 3s 6ms/step - loss: 0.4605 - val_loss: 1.4285
Epoch 286/400
583/588 [============================>.] - ETA: 0s - loss: 0.4587
Epoch 00286: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4595 - val_loss: 1.4522
Epoch 287/400
584/588 [============================>.] - ETA: 0s - loss: 0.4583
Epoch 00287: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4598 - val_loss: 1.4244
Epoch 288/400
587/588 [============================>.] - ETA: 0s - loss: 0.4558
Epoch 00288: val_loss did not improve from 1.35089
588

587/588 [============================>.] - ETA: 0s - loss: 0.4512
Epoch 00321: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4523 - val_loss: 1.4563
Epoch 322/400
584/588 [============================>.] - ETA: 0s - loss: 0.5149
Epoch 00322: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.5167 - val_loss: 1.5390
Epoch 323/400
584/588 [============================>.] - ETA: 0s - loss: 0.5123
Epoch 00323: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.5133 - val_loss: 1.3646
Epoch 324/400
582/588 [============================>.] - ETA: 0s - loss: 0.4741
Epoch 00324: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4747 - val_loss: 1.4268
Epoch 325/400
587/588 [============================>.] - ETA: 0s - loss: 0.4557
Epoch 00325: val_loss did not improve from 1.35089
588

583/588 [============================>.] - ETA: 0s - loss: 0.4348
Epoch 00358: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4361 - val_loss: 1.3865
Epoch 359/400
586/588 [============================>.] - ETA: 0s - loss: 0.4363
Epoch 00359: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4366 - val_loss: 1.4130
Epoch 360/400
584/588 [============================>.] - ETA: 0s - loss: 0.4357
Epoch 00360: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4360 - val_loss: 1.3745
Epoch 361/400
586/588 [============================>.] - ETA: 0s - loss: 0.4352
Epoch 00361: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4356 - val_loss: 1.4012
Epoch 362/400
585/588 [============================>.] - ETA: 0s - loss: 0.4351
Epoch 00362: val_loss did not improve from 1.35089
588

584/588 [============================>.] - ETA: 0s - loss: 0.4301
Epoch 00395: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4320 - val_loss: 1.4405
Epoch 396/400
587/588 [============================>.] - ETA: 0s - loss: 0.4343
Epoch 00396: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4353 - val_loss: 1.4309
Epoch 397/400
587/588 [============================>.] - ETA: 0s - loss: 0.4280
Epoch 00397: val_loss did not improve from 1.35089
588/588 [==============================] - 3s 6ms/step - loss: 0.4291 - val_loss: 1.4228
Epoch 398/400
587/588 [============================>.] - ETA: 0s - loss: 0.4241
Epoch 00398: val_loss did not improve from 1.35089
588/588 [==============================] - 4s 6ms/step - loss: 0.4252 - val_loss: 1.4045
Epoch 399/400
584/588 [============================>.] - ETA: 0s - loss: 0.4229
Epoch 00399: val_loss did not improve from 1.35089
588

In [70]:
training_data[1]

('company ha pretty prevalent good old boy culture',
 array([0., 0., 0., 0., 0., 8., 0., 0.]))